# Stream Ciphers

In [1]:
def sum(a, b):
    # Suma modulo 2^32
    return (a + b) & ((1 << 32) - 1)

In [6]:
def str_to_hex(s): return hex(int(s.encode().hex()))

# Salsa

### Pad

In [11]:
# const, 32 bits
c0 = "expa"
c1 = "nd 3"
c2 = "2-by"
c3 = "te k"

# TODO: string to hex 

def pad(s, j, n):
    # s: seed (256 bits), 32 bits word s0, s1, s2, ..., s7 
    # j: counter (64 bits)
    # n: nonce (64 bits)
    # returns 512 bits
    return [
        c0, s[0], s[1], s[2],
        s[3], c1, n[0], n[1],
        j[0], j[1], c2, s[4],
        s[5], s[6], s[7], c3
    ]

### Permutation $\pi$

Interating a simple permutation a fized number of times

In [3]:
def rot(w, r): 
    return ((w << r) & ((1 << 32) - 1)) | (w >> (32 - r)) 

In [3]:
# Quater tound 
def QR_salsa(a, b, c, d): 
    b = b ^ rot(sum(a, d), 7)
    c = c ^ rot(sum(b, a), 9)
    d = d ^ rot(sum(c, b), 13)
    a = a ^ rot(sum(d, c), 18)
    
    return a, b, c, d

In [8]:
ROUNDS = 8

def perm_salsa(x): 
    # x: 512 bits 
    # return 512 bits
    
    i = x.copy()
    
    for j in range(0, ROUNDS, 2): 
        # Odd round
        i[0], i[4], i[8], i[12] = QR_salsa(i[0], i[4], i[8], i[12]) # Col1
        i[5], i[9], i[13], i[1] = QR_salsa(i[5], i[9], i[13], i[1]) # Col2
        i[10], i[14], i[2], i[6] = QR_salsa(i[10], i[14], i[2], i[6]) # Col3
        i[15], i[3], i[7], i[11] = QR_salsa(i[15], i[3], i[7], i[11]) # Col4
        # Even round 
        i[0], i[1], i[2], i[3] = QR_salsa(i[0], i[1], i[2], i[3]) # Row1
        i[5], i[6], i[7], i[4] = QR_salsa(i[5], i[6], i[7], i[4]) # Row2
        i[10], i[11], i[8], i[9] = QR_salsa(i[10], i[11], i[8], i[9]) # Row3
        i[15], i[12], i[13], i[14] = QR_salsa(i[15], i[12], i[13], i[14]) # Row 4
    return i

### Generador psudoaletorio

In [35]:

L = 1 << 63

def G(s, n):
    # Seed: 256 bits
    # n: 64 bits 
    
    r = []
    for j in range(L): # j: 64 bits 
        h_j = pad(s, j, n) 
        perm_j = perm_salsa(h_j)

        r_j = []

        for i in range(16): 
            r_j[i] = perm_j[i] + h_j[i]
        r.append(r_j)
        
    return r

### Salsa 

In [ ]:
s = None
n = None 
k = G(s, n)

## ChaCha

In [51]:
def QR_chacha(a, b, c, d):
    a = sum(a, b)
    d = d ^ a 
    d = rot(d, 16)

    c = sum(c, d)
    b = b ^ c
    b = rot(b, 12)

    a = sum(a, b)
    
    return a, b, c, d